In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import gc
gc.collect()

35

In [3]:
data = pd.read_csv('./input/lending-club-loan-data/loan_enhanced.csv', low_memory=False)
data.head(5)
data = data.drop(['Unnamed: 0'],axis=1)
data = data.fillna(0)

### Approach to Scaling & Encoding:

1. Drop categorical columns from the dataframe
2. Scale rest of the columns
3. Add the dropped columns back in the dataframe
4. Encode the categorical columns

In [4]:
data_continous_features = data.drop(['last_pymnt_d','next_pymnt_d','last_credit_pull_d','term','grade','home_ownership',
                              'verification_status','pymnt_plan','initial_list_status','application_type',
                              'disbursement_method','debt_settlement_flag','Loan_Category','Loan_Purpose',
                              'Interest_Rate_Slabs','region'], axis=1)

data_cat_features = data[['last_pymnt_d','next_pymnt_d','last_credit_pull_d','term','grade','home_ownership',
                              'verification_status','pymnt_plan','initial_list_status','application_type',
                              'disbursement_method','debt_settlement_flag','Loan_Category','Loan_Purpose',
                              'Interest_Rate_Slabs','region']]

In [5]:
print (data_continous_features.shape)
print (data_cat_features.shape)

(2260668, 84)
(2260668, 16)


In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

print ('Starting scaling of numeric features..')

#data_scaled = scaler.fit_transform(data[['funded_amnt','annual_inc','loan_amnt','funded_amnt_inv','installment','revol_bal','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_amnt','policy_code','last_pymnt_d_year','next_pymnt_d_year','last_credit_pull_d_year','dti','revol_util','tot_cur_bal','total_bal_il','max_bal_bc','total_rev_hi_lim','avg_cur_bal','bc_open_to_buy','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit','issue_year']])
#data_scaled_df = pd.DataFrame(data=data_scaled, columns=['funded_amnt','annual_inc','loan_amnt','funded_amnt_inv','installment','revol_bal','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_amnt','policy_code','last_pymnt_d_year','next_pymnt_d_year','last_credit_pull_d_year','dti','revol_util','tot_cur_bal','total_bal_il','max_bal_bc','total_rev_hi_lim','avg_cur_bal','bc_open_to_buy','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit','issue_year'])
#reduced_data = data.drop(['funded_amnt','annual_inc','loan_amnt','funded_amnt_inv','installment','revol_bal','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_amnt','policy_code','last_pymnt_d_year','next_pymnt_d_year','last_credit_pull_d_year','dti','revol_util','tot_cur_bal','total_bal_il','max_bal_bc','total_rev_hi_lim','avg_cur_bal','bc_open_to_buy','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit','issue_year'],axis=1)
#data_scaled = pd.concat([data_scaled_df, reduced_data],axis=1)
#data_scaled = pd.DataFrame(data_scaled, columns=data.columns)

data_continous_features_array = scaler.fit_transform(data_continous_features)
data_continous_features = pd.DataFrame(data_continous_features_array, columns=data_continous_features.columns)
print ('Ending scaling of numeric features..')


Starting scaling of numeric features..


C:\MachineLearning\anaconda\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\MachineLearning\anaconda\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


Ending scaling of numeric features..


In [7]:
def label_encode_features (data, feature):
    tmp_feature = pd.get_dummies(data[feature], prefix = feature, prefix_sep='_')
    tmp_feature_df = pd.DataFrame(data=tmp_feature)

    data_tmp = pd.concat([data, tmp_feature_df],axis=1)
    return data_tmp.drop([feature],axis=1)

In [8]:
print ('Starting to encode categorical features...')
print (data_cat_features.shape)
data_cat_features = label_encode_features(data_cat_features, 'term')
data_cat_features = label_encode_features(data_cat_features, 'grade') 
data_cat_features = label_encode_features(data_cat_features, 'home_ownership')
data_cat_features = label_encode_features(data_cat_features, 'verification_status')
data_cat_features = label_encode_features(data_cat_features, 'pymnt_plan')
data_cat_features = label_encode_features(data_cat_features, 'initial_list_status')
data_cat_features = label_encode_features(data_cat_features, 'application_type')
data_cat_features = label_encode_features(data_cat_features, 'disbursement_method')
data_cat_features = label_encode_features(data_cat_features, 'debt_settlement_flag')
data_cat_features = label_encode_features(data_cat_features, 'Loan_Category')
data_cat_features = label_encode_features(data_cat_features, 'Loan_Purpose')
data_cat_features = label_encode_features(data_cat_features, 'Interest_Rate_Slabs')
data_cat_features = label_encode_features(data_cat_features, 'region')
print ('Ended encoding categorical features...')
print (data_cat_features.shape)

Starting to encode categorical features...
(2260668, 16)
Ended encoding categorical features...
(2260668, 44)


In [9]:
def convert_data_to_month_year(data, feature):
    tmp_series = pd.to_datetime(data[feature])
    mth = feature+'_month'
    yr = feature+'_year'
    data[mth] = tmp_series.dt.month
    data[yr] = tmp_series.dt.year
    return data

print ('Starting to expand date features ..')
data_cat_features = convert_data_to_month_year(data_cat_features, 'last_pymnt_d')
data_cat_features = convert_data_to_month_year(data_cat_features, 'next_pymnt_d')
data_cat_features = convert_data_to_month_year(data_cat_features, 'last_credit_pull_d')

print ('Done expanding, now starting to drop old date features')

data_cat_features = data_cat_features.drop(['last_pymnt_d','next_pymnt_d','last_credit_pull_d'], axis=1)

Starting to expand date features ..
Done expanding, now starting to drop old date features


In [10]:
data_scaled = pd.concat([data_cat_features, data_continous_features],axis=1)

In [12]:
data_scaled.to_csv('./input/lending-club-loan-data/loan_encoded.csv')

In [ ]:
import gc
gc.collect()